In [2]:

from collections import deque

import hashlib

In [30]:
class Nodo():
    def __init__(self):

        self.hash = None
        self.is_leaf = False
        self.left_child = None
        self.right_child = None
        self.parent = None
        self.value = 0
        self.side = None
        self.brother = None

    def __repr__(self):
        return str(self.hash)

class MerkleTree:
    def __init__(self, strings, hash):
        self.strings = strings
        self.hash = hash
        self.hojas = None
        self.altura = self.get_altura()
        self.insert()
        self.hashear()
    
    def get_altura(self):
        for i in range(len(self.strings)):
            if 2**i >= len(self.strings):
                self.altura = i
                return self.altura
            else:
                pass


    def hashear(self):
        value = self.strings[-1]
        # print(value)
        # print(value)
        for i in range(len(self.hojas)):
            if i >= len(self.strings)-1:
                self.hojas[i].hash = self.hash(value)
            else:
                self.hojas[i].hash = self.hash(self.strings[i])
            

        cola = []
        for i in range(0,len(self.hojas),2):
            cola.append(self.hojas[i].parent)
        # print(self.hojas)
        # print(cola)

        count = 0
        while count <= self.altura-1:

            cola_aux = []
            for i in range(0,len(cola)):
                cola[i].hash = str(self.hash(str(cola[i].left_child.hash) + str(cola[i].right_child.hash)))
                # print(cola[i].hash)
                if i%2 != 0:
                    cola_aux.append(cola[i].parent)
            # print([cola])
            cola = cola_aux
            count += 1

        

        # return self.hojas
        
    def insert(self):
        count = 0
        value = 0
        root = Nodo()
        cola = []
        cola.append(root)
        self.hojas = []
        # print(cola)
        while count <= self.altura:
            if count == self.altura:
                for i in cola:
                    i.left_child = None
                    i.right_child = None
                    i.is_leaf = True
                    i.value = value
                    value += 1
                for i in range(len(cola)):
                    if i < len(cola)-1:
                        cola[i].next_leaf = cola[i+1]
                for i in cola:
                    self.hojas.append(i)
                break
            else:
                cola_aux = []
                for i in cola:
                    i.left_child = Nodo()
                    i.right_child = Nodo()
                    i.left_child.parent = i
                    i.right_child.parent = i
                    i.left_child.brother = i.right_child 
                    i.right_child.brother = i.left_child
                    i.right_child.side = 'd'
                    i.left_child.side = 'i'
                    i.is_leaf = False
                    i.value = value
                    value += 1
                    cola_aux.append(i.left_child)
                    cola_aux.append(i.right_child)
                for i in range(len(cola)):
                    if i < len(cola)-1:
                        cola[i].next_leaf = cola[i+1]
                cola = cola_aux
                # print(cola)
            count += 1
        

    
    def get_root(self):
        count = 0
        cola = [self.hojas[0]]
        while count <= self.altura:
            if count == self.altura:
                # print(cola[0])
                return cola[0].hash
            else:
                cola_aux = []
                cola_aux.append(cola[0].parent)
                cola = cola_aux
            count += 1
    
    def get_proof_for(self,string):
        lista_return = []
        cola = []
         
        # print('hash buscado',self.hash(string))
        for i in self.hojas:
            # print(i)
            # print('hash hoja',i.hash)
            if self.hash(string) == i.hash:
                cola.append(i)
                lista_return.append((i.brother.hash,i.brother.side))

        count = 0
        while count <= self.altura-1:
            if count == self.altura-1:
                break
            else:
                cola_aux = []
                for i in cola:
                    lista_return.append(( i.parent.brother.hash,i.parent.brother.side))
                    cola_aux.append(i.parent)
                    cola = cola_aux
            count += 1


        if len(lista_return) == 0:
            return None
        return lista_return
    


In [47]:
def verify(raiz,string,proof,hash):
    # print(string)
    item_hash = hash(string)
    next_hash = None

    for i in proof:
        if i[1] == 'i':
            item_hash = str(hash(str(i[0]+str(item_hash))))
        else:
            item_hash = str(hash(str(item_hash)+str(i[0])))
        next_hash = item_hash
    # print(next_hash)
    # print(raiz)
    if next_hash == raiz:
        return True
    else:
        return False